# **Análisis del uso de las energías en España y su relación con las energías renovables.**

### Trabajo por **Álvaro Mendoza González** y **Jesús Enrique Cartas Rascón**


---


En este análisis pretendemos entender qué energías se utlizan en España principalmente, cuánto y cómo. Intentaremos también arrojar luz en el tema de las energías renovables y su relación con las energías clásicas, para ver en qué medida merece la pena dar el salto. 

Todos los datos provienen del [**Eurostat**](https://ec.europa.eu/eurostat), una gran fuente de datos estadísticos a nivel europeo.




# Índice

1. <a href=#intro>Introducción</a>

2. <a href=#howandwhy>Cómo y cuánto se usa cada tipo de energía</a>

3. <a href=#access>Cómo de accesible es esta energía a los diferentes sectores</a>

4. <a href=#efficient>Relación producción/consumo de energía</a>

5. <a href=#renew>Cómo evoluciona el ritmo de uso de las energías renovables</a>

6. <a href=#relation>Cómo se relacionan éstas energías con las energías renovables</a>

## 1. Introducción <a name='intro' />

Dedicaremos esta primera parte a importar las librerías y archivos necesarios para el análisis, así como a definir un conjunto de funciones que nos sirvan de ayuda y agilicen el proceso de desarrollo.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
base_url = "https://raw.githubusercontent.com/jesi-rgb/proyectoTGI/master/"

In [ ]:
electricity_prices_household = pd.read_csv(base_url + "energy_data/electricity_prices_household.csv", delimiter=';')
gas_prices_household = pd.read_csv(base_url + "energy_data/gas_prices_household.csv", delimiter=';')

In [ ]:
def consulta_por_valor_columna(dataframe, valor, nombre_columna):
    return dataframe[dataframe[nombre_columna] == valor]

In [1]:
def crear_grafica(x_data, y_data, titulo='XD', x_label='X', y_label='D'):
    '''Crea una gráfica simple a partir de un conjunto de datos.'''
    plt.figure(figsize=(20, 10))
    plt.plot(x_data, y_data)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(titulo)

In [ ]:
def castear_a_float(input_list):
    '''Recibe una lista de valores y devuelve otra con los valores interpretados como flotantes.
    Esta función NO edita la columna, devuelve una nueva. La columna del dataframe original deberá ser sobreescrita pertinentemente.
    ''' 
    return input_list.apply(float)

In [ ]:
def string_replace_columna(columna, valor_a_buscar, valor_sustituto):
    '''Busca espacios en la columna y los elimina
    Esta función NO edita la columna, devuelve una nueva. La columna del dataframe original deberá ser sobreescrita pertinentemente.
    '''
    return columna.str.replace(valor_a_buscar, valor_sustituto)

In [ ]:
def eliminar_valores_invalidos(dataframe, columna, valor_invalido, valor_nuevo):
    '''Sustituye las instancias de valor_invalido por las de valor nuevo en la columna. Por ejemplo, para sustituir \':\' por -1
    Es recomendable pasar el valor nuevo como tipo igual al que tiene el data frame, para que luego los casteos no se hagan complicados
    '''
    dataframe.loc[dataframe[columna] == valor_invalido, columna] = valor_nuevo

In [ ]:
text_arguments = {
    "fontfamily":["Consolas", "Inconsolata"],
    "fontsize":18
}

In [ ]:
tableau20 = np.array([(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),    
             (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),    
             (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),    
             (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),    
             (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)]) 
tableau20 = tableau20/255

In [2]:
def annot_max(x,y, ax=None, xPos=1, yPos=1):
    '''Indica cuál es el máximo del conjunto de datos que pasemos con una etiquetita dentro de una gráfica'''
    xmax = x[pd.Series.idxmax(y)]
    ymax = y.max()
    text = "Max: x={0:s}, y={1:.3f}".format(xmax, float(ymax))
    if not ax:
        ax=plt.gca()
    bbox_props = dict(boxstyle="square", pad=0.3, fc="w" , ec="k", lw=0.52)
    arrowprops = dict(arrowstyle= "-|>", connectionstyle="angle, angleA=0, angleB=60")
    kw = dict(xycoords='data',textcoords="axes fraction",
              arrowprops=arrowprops, bbox=bbox_props, ha="right", va="top" )
    ax.annotate(text, xy=(xmax, ymax), xytext=(xPos, yPos), **kw)

# 2. Cómo y cuánto se usa cada tipo de energía <a name='howandwhy' />

En esta sección trataremos de hallar las proporciones de uso de las energías más importantes en España: la energía eléctrica, el consumo de gas, y otras como petróleo o nucleares.

# 3. Cómo de accesible es esta energía a los diferentes sectores <a name='access' />

Una vez visto qué es lo que más se consume, vamos a ver cuánto cuesta consumirlo. Trataremos de ver cuánto le cuesta la energía a los distintos sectores que la demandan, como el de viviendas (usuarios normales), empresas y demás, y ver si los precios son realmente accesibles.

In [ ]:
electricity_prices_household_es = consulta_por_valor_columna(electricity_prices_household, 'ES', 'geo\\time')
gas_prices_household_es = consulta_por_valor_columna(gas_prices_household, 'ES', 'geo\\time')

In [ ]:
electricity_prices_household_es_eur = consulta_por_valor_columna(electricity_prices_household_es, 'EUR', 'currency')
electricity_prices_household_es_eur_xtax = consulta_por_valor_columna(electricity_prices_household_es_eur, 'X_TAX', 'tax')


gas_prices_household_es_eur = consulta_por_valor_columna(gas_prices_household_es, 'EUR', 'currency')
gas_prices_household_es_eur_xtax = consulta_por_valor_columna(gas_prices_household_es_eur, 'X_TAX', 'tax')

In [ ]:
electricity_prices_household_es_eur_xtax = electricity_prices_household_es_eur_xtax.melt(
    id_vars=['product', 'consom', 'unit', 'tax', 'currency', 'geo\\time'],
    var_name='Semestres',
    value_name='Precio' 
)

In [ ]:
gas_prices_household_es_eur_xtax = gas_prices_household_es_eur_xtax.melt(
    id_vars=['product', 'consom', 'unit', 'tax', 'currency', 'geo\\time'],
    var_name='Semestres',
    value_name='Precio' 
)

In [ ]:
eliminar_valores_invalidos(electricity_prices_household_es_eur_xtax, 'Precio', ": ", np.NaN)
eliminar_valores_invalidos(electricity_prices_household_es_eur_xtax, 'Precio', ": u", np.NaN)

electricity_prices_household_es_eur_xtax.Precio = string_replace_columna(electricity_prices_household_es_eur_xtax.Precio, 'p', "")
electricity_prices_household_es_eur_xtax.Precio = string_replace_columna(electricity_prices_household_es_eur_xtax.Precio, " ", "")

In [ ]:
electricity_prices_household_es_eur_xtax.Precio = castear_a_float(electricity_prices_household_es_eur_xtax.Precio)

In [ ]:
eliminar_valores_invalidos(gas_prices_household_es_eur_xtax, 'Precio', ": ", np.NaN)
eliminar_valores_invalidos(gas_prices_household_es_eur_xtax, 'Precio', ": u", np.NaN)

gas_prices_household_es_eur_xtax.Precio = string_replace_columna(gas_prices_household_es_eur_xtax.Precio, 'p', "")
gas_prices_household_es_eur_xtax.Precio = string_replace_columna(gas_prices_household_es_eur_xtax.Precio, " ", "")

In [ ]:
gas_prices_household_es_eur_xtax.Precio = castear_a_float(gas_prices_household_es_eur_xtax.Precio)

In [ ]:
electricity_prices_household_es_eur_xtax = electricity_prices_household_es_eur_xtax[electricity_prices_household_es_eur_xtax['Semestres'] != '2007S1']

In [ ]:
less_1k = (consulta_por_valor_columna(electricity_prices_household_es_eur_xtax, 4161901, 'consom')).Precio
from_1k_to_2k = (consulta_por_valor_columna(electricity_prices_household_es_eur_xtax, 4161902, 'consom')).Precio
from_2k_to_5k = (consulta_por_valor_columna(electricity_prices_household_es_eur_xtax, 4161903, 'consom')).Precio
from_5k_to_15k = (consulta_por_valor_columna(electricity_prices_household_es_eur_xtax, 4161904, 'consom')).Precio
over_15k = (consulta_por_valor_columna(electricity_prices_household_es_eur_xtax, 4161905, 'consom')).Precio

In [ ]:
semestres_elec = (electricity_prices_household_es_eur_xtax.Semestres).drop_duplicates()

# 4. Relación producción/consumo de energía <a name='efficient' />

En esta sección vamos a estudiar cuánto se produce y consume de cada energía y trataremos de encontrar la relación entre las mismas y concluir algo en base a los datos obtenidos anteriormente.

# 6. Cómo evoluciona el ritmo de uso de las energías renovables <a name='renew' />

# 7. Cómo se relacionan éstas energías con las energías renovables <a name='relation' />